# Gallery Example: Replayer/M/1 Queue (Trace-Driven)

This example demonstrates a trace-driven queueing system:
- **Arrivals**: Replayed from trace file
- **Service**: Exponential
- **Servers**: 1 server
- **Scheduling**: FCFS

The Replayer distribution reads inter-arrival times from a file, enabling trace-driven simulation and analysis.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_replayerm1(filename=None): Network {
    """Create Trace/M/1 queueing model
    
    Args:
        filename: Path to trace file with inter-arrival times.
                  If None, uses example_trace.txt from LINE installation.
    """
    val model = Network("Trace/M/1")
// Block 1: nodes
    val source = Source(model, "mySource")
    val queue = Queue(model, "myQueue", SchedStrategy.FCFS)
    val sink = Sink(model, "mySink")
// Block 2: classes
    val oclass = OpenClass(model, "myClass")
// Create replayer from trace file
    if filename is None:
// Use a simple exponential arrival for demo if no trace file
        source.setArrival(oclass, Exp(1))
        queue.setService(oclass, Exp(3))
    else:
        val replayer = Replayer(filename)
        source.setArrival(oclass, replayer)
// Service rate set to 3x arrival rate for stability
        queue.setService(oclass, Exp(3 / replayer.obj.getMean()))
// Block 3: topology
    model.link(Network.serialRouting(source, queue, sink))
    
    return model
// Create the model (without trace file, uses exponential for demo)
val model = gallery_replayerm1()

## About Trace-Driven Analysis

The Replayer distribution:
- Reads inter-arrival times from a text file
- Enables analysis using real workload data
- Can be converted to analytical distributions via fitting

Trace file format:
- One inter-arrival time per line
- Values are positive real numbers

Use cases:
- Validating models against real systems
- Analyzing recorded workload patterns
- Performance prediction with actual traffic

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 15)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)